In [ ]:
import syft as sy
sy.__version__

In [ ]:
import inspect
from inspect import _signature_fromstr
import types
import re
from types import FunctionType, LambdaType, ModuleType
from typing import Callable

import numpy as np

# func_signature = signature(np.array) # no signature found for builtin <built-in function array>

# func_signature = signature(np.all) # numpy._globals._NoValueType not in TYPE_BANK

func_signature = inspect.signature(np.zeros_like)
display(func_signature)
func_signature2 = inspect.signature(np.all)
display(func_signature2)
# sy.serialize(func_signature2)

In [ ]:
function_signatures_registry = {
    "concatenate": "concatenate(a1,a2, *args,axis=0,out=None,dtype=None,casting='same_kind')",
    "set_numeric_ops": "set_numeric_ops(op1,op2, *args)",
    "geterrorobj": "geterrobj()",
    "source": "source(object, output)",
    "fromstring": "fromstring(string, dtype=None, count=-1, like=None)"
}

def get_signature_from_docstring(doc: str, callable_name: str) -> str:
    if not doc or callable_name not in doc:
        return None
    else:
        doc = re.sub(r'\s',"", doc.split('\n\n')[0])
        search_res = re.search(rf"{callable_name}\((.+)\)", doc)
        if search_res:
            signature = search_res.group(1)
            # decomposing "[]" optional  params
            params = re.findall(r"\[(.+?)\]", signature)
            if params:
                print(callable_name)
                for param in params[:-1]:
                    signature = signature.replace(f"[{param}]", param)
            
                if re.search(fr"(?<={params[-1]})\],", signature):
                    signature = signature.replace(f"[{params[-1]}],", params[-1])
                else:
                    signature = signature.replace(f"[{params[-1]}]",  f', {",".join([f"{param}=None" for param in params[-1].split(",") if param])}')

            signature = re.sub(r",(\/|\*)", "", signature)
            signature = re.sub(r"dtype=(\w+),", "dtype=None,", signature)
            return f"{callable_name}({signature})"
        else:
            return None

def get_signature_from_registry(callable_name: str) -> str:
    return function_signatures_registry[callable_name]


def generate_signature(_callable) -> inspect.Signature:
    name, doc = _callable.__name__, _callable.__doc__
    # returning predefined signature if in signature registry
    name_in_registry = name in function_signatures_registry.keys()
    text_signature = get_signature_from_registry(name) \
        if name_in_registry else  get_signature_from_docstring(doc, name)
    # TODO safe handling if function signature can not be generated
    text_signature = "()" if text_signature is None else  text_signature  
    return _signature_fromstr(inspect.Signature, _callable, text_signature, True)


def generate_signature_2(_callable):
    pass


def get_signature(_callable) -> inspect.Signature:
    try:
        res = inspect.signature(_callable)
        if res is None or "OutStream" in str(res):
            raise ValueError("")
        else:
            return res
    except Exception:
        return generate_signature(_callable)

In [ ]:



for lib_module in [np]:
    parent_path = lib_module.__name__
    for attr in dir(lib_module):
        lib_obj = getattr(lib_module, attr)
        path = f"{parent_path}.{attr}"  
        if inspect.isfunction(lib_obj) or (callable(lib_obj) and not inspect.isclass(lib_obj)):
            signature = get_signature(lib_obj)
            try:
                sy.serialize(signature)
            except Exception as e:
                print(f"skipping {lib_obj} with signature: {signature}")
                pass

In [ ]:
def get_signature_from_docstring(doc: str, callable_name: str) -> str:
    if not doc or callable_name not in doc:
        return None
    else:
        doc = re.sub(r'\s',"", doc.split('\n\n')[0])
        search_res = re.search(rf"{callable_name}\((.+)\)", doc)
        if search_res:
            signature = search_res.group(1)
            # decomposing "[]" optional  params
            params = re.findall(r"\[(.+?)\]", signature)
            if params:
                for param in params[:-1]:
                    signature = signature.replace(f"[{param}]", param)
            
                if re.search(fr"(?<={params[-1]})\],", signature):
                    signature = signature.replace(f"[{params[-1]}],", params[-1])
                else:
                    signature = signature.replace(f"[{params[-1]}]",  f', {",".join([f"{param}=None" for param in params[-1].split(",") if param])}')

            signature = re.sub(r",(\/|\*)", "", signature)
            signature = re.sub(r"dtype=(\w+),", "dtype=None,", signature)
            return f"{callable_name}({signature})"
        else:
            return None

_callable = np.test        
sig = get_signature_from_docstring(_callable.__doc__, _callable.__name__)
sig

In [ ]:
display(_callable)
signature = get_signature(_callable)
display(signature)

sy.serialize(signature)

In [ ]:
# inspect.signature(np.test)
inspect.signature(np.source)

In [ ]:
# np._pytesttester.PytestTester
check_funciton = lambda func: func if (inspect.isfunction(getattr(np, func)) or ((callable(getattr(np, func))) and not inspect.isclass(getattr(np, func)))) else None
[x for x in list(map(check_funciton, dir(np))) if x is not None]
# inspect.isfunction(np.all)

In [ ]:
_callable = np.fromstring
inspect._signature_fromstr(inspect.Signature, _callable, "fromstring(string, dtype=None, count=-1, like=None)")

In [ ]:
sig = "string, dtype=None, count=-1, sep, like=None"
optional = False
params = sig.split(', ')
for idx in range(len(params)):
    match = re.match(r"(\w+)=(\S+)", params[idx])
    if not optional and match:
        optional=True
    elif optional and not match:
        params[idx]=f"{params[idx]}=None"
    else:
        continue
    
        

params   

In [ ]:
inspect.signature(np.zeros_like)

In [ ]:
", ".join([f"{param}=None" for param in ", signature, extobj".split(", ") if param])

In [ ]:
sig_str = get_text_signature(np.concatenate.__doc__)


inspect._signature_fromstr(inspect.Signature, np.arange, "()", True)

In [ ]:
# inspect.signature(inspect.Parameter.replace)
inspect.signature(np.concatenate).parameters

In [ ]:
custo_arrange = np.arange
custo_arrange.__signature__ = inspect._signature_fromstr(inspect.Signature, np.arange, "arange(start, stop, step, dtype=None, like=None)", True)
